# SUMÁRIO

- [1. INFOS API DATAJUD](#1-infos-api-datajud)

- [EXEMPLOS DE CONSULTA INDIVIDUAL](#testes-consulta-individual)

### 1. INFOS API DATAJUD

#### ENDPOINT

- https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search

#### TESTES CONSULTA INDIVIDUAL

[VOLTAR AO SUMÁRIO](#sumário)

### 1. CARREGAR PACOTES 

In [2]:
import requests
import json
import ast

import pandas as pd
pd.set_option('display.max_rows', 100) 
import numpy as np

import concurrent.futures
from time import sleep
from tqdm import tqdm

In [3]:
API_KEY = "cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw=="
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"
headers = {
    'Authorization': f'ApiKey {API_KEY}',
    'Content-Type': 'application/json'
}

def consultar_processo(numero):
    payload = json.dumps({
        "query": {
            "match": {
                "numeroProcesso": numero
            }
        }
    })

    try:
        response = requests.post(url, headers=headers, data=payload, timeout=30)
        response.raise_for_status()
        data = response.json()

        resultados = []
        for hit in data.get("hits", {}).get("hits", []):
            registro = hit["_source"]
            registro["_id"] = hit.get("_id")
            resultados.append(registro)

        return resultados

    except Exception as e:
        print(f"❌ Erro no processo {numero}: {e}")
        return []

def processar_em_lotes(lista, max_threads=5):
    todos_resultados = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = {executor.submit(consultar_processo, numero): numero for numero in lista}

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            resultado = future.result()
            todos_resultados.extend(resultado)
            sleep(0.1)  

    return todos_resultados


In [4]:
lote001 = pd.read_parquet('./app10/resultado-api/lotes/lote_001.parquet')
# lote1200 = pd.read_parquet('./app10/resultado-api/lotes/lote_1200.parquet')

In [7]:
lote001['movimentos'][0]

array([{'codigo': 26, 'complementosTabelados': array([{'codigo': 2, 'descricao': 'tipo_de_distribuicao_redistribuicao', 'nome': 'sorteio', 'valor': 2}],
             dtype=object), 'dataHora': '2024-07-16T17:12:02.000Z', 'nome': 'Distribuição'}                                                             ,
       {'codigo': 581, 'complementosTabelados': array([{'codigo': 4, 'descricao': 'tipo_de_documento', 'nome': 'Outros documentos', 'valor': 80}],
             dtype=object), 'dataHora': '2024-07-16T17:18:33.000Z', 'nome': 'Documento'}                                                          ,
       {'codigo': 11383, 'complementosTabelados': None, 'dataHora': '2024-07-17T10:22:21.000Z', 'nome': 'Ato ordinatório'},
       {'codigo': 60, 'complementosTabelados': array([{'codigo': 4, 'descricao': 'tipo_de_documento', 'nome': 'Certidão', 'valor': 107}],
             dtype=object), 'dataHora': '2024-07-17T10:22:36.000Z', 'nome': 'Expedição de documento'}                                   

In [5]:
df = pd.DataFrame(consultar_processo('10203815120248260008'))

df

,numeroProcesso,classe,sistema,formato,tribunal,dataHoraUltimaAtualizacao,grau,@timestamp,dataAjuizamento,movimentos,id,nivelSigilo,orgaoJulgador,assuntos,_id
0,10203815120248260008,"{'codigo': 7, 'nome': 'Procedimento Comum Cível'}","{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2025-02-06T01:29:47.490Z,G1,2025-02-10T17:58:51.204459492Z,2024-11-26T15:26:25.000Z,"[{'complementosTabelados': [{'codigo': 2, 'val...",TJSP_G1_10203815120248260008,0,"{'codigo': 10468, 'nome': '04 CIVEL DE TATUAPE'}","[{'codigo': 7779, 'nome': 'Indenização por Dan...",TJSP_G1_10203815120248260008


In [6]:
df['movimentos'][0]

[{'complementosTabelados': [{'codigo': 2,
    'valor': 2,
    'nome': 'sorteio',
    'descricao': 'tipo_de_distribuicao_redistribuicao'}],
  'codigo': 26,
  'nome': 'Distribuição',
  'dataHora': '2024-11-26T15:36:33.000Z'},
 {'complementosTabelados': [{'codigo': 4,
    'valor': 80,
    'nome': 'Outros documentos',
    'descricao': 'tipo_de_documento'}],
  'codigo': 581,
  'nome': 'Documento',
  'dataHora': '2024-11-26T16:27:42.000Z'},
 {'complementosTabelados': [{'codigo': 4,
    'valor': 80,
    'nome': 'Outros documentos',
    'descricao': 'tipo_de_documento'}],
  'codigo': 581,
  'nome': 'Documento',
  'dataHora': '2024-11-26T16:28:08.000Z'},
 {'complementosTabelados': [{'codigo': 4,
    'valor': 80,
    'nome': 'Outros documentos',
    'descricao': 'tipo_de_documento'}],
  'codigo': 581,
  'nome': 'Documento',
  'dataHora': '2024-11-26T16:28:45.000Z'},
 {'complementosTabelados': [{'codigo': 4,
    'valor': 80,
    'nome': 'Outros documentos',
    'descricao': 'tipo_de_documento'}],

#### 2. Função para expandir colunas com dicionários

In [7]:
def expandir_colunas(df, colunas_alvo, chaves=['codigo', 'nome'], como_string=True, separador='|'):
    """
    Expande colunas contendo listas de dicionários (ou dicts), criando novas colunas para cada chave.
    Os valores podem ser retornados como listas ou strings separadas por vírgula (default).
    Mantém a ordem original das colunas, inserindo as novas logo após a original.
    """
    df_exp = df.copy()
    novas_colunas = {}

    def extrair_primeiros(lista_ou_dict, chave):
        if isinstance(lista_ou_dict, dict):
            val = lista_ou_dict.get(chave, None)
            return str(val) if como_string else val
        elif isinstance(lista_ou_dict, list):
            try:
                valores = [str(d.get(chave)) for d in lista_ou_dict if isinstance(d, dict)]
                return separador.join(valores) if como_string else valores
            except:
                return None
        return None

    for col in colunas_alvo:
        if col not in df_exp.columns:
            continue
        valores_expandidos = df_exp[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
        for chave in chaves:
            nova_coluna = f"{col}_{chave}"
            # Garante que só cria a coluna uma vez
            if nova_coluna not in novas_colunas:
                novas_colunas[nova_coluna] = valores_expandidos.apply(lambda v: extrair_primeiros(v, chave))

    # Insere as novas colunas logo após cada coluna original
    for col in colunas_alvo:
        if col not in df_exp.columns:
            continue
        idx_col = df_exp.columns.get_loc(col)
        insert_at = idx_col + 1
        for chave in chaves:
            nova_coluna = f"{col}_{chave}"
            if nova_coluna in df_exp.columns:
                continue  # Não sobrescreve se já existe
            df_exp.insert(insert_at, nova_coluna, novas_colunas[nova_coluna])
            insert_at += 1

    if 'numeroProcesso' in df_exp.columns:
        cols = df_exp.columns.tolist()
        cols.remove('numeroProcesso')
        df_exp = df_exp[['numeroProcesso'] + cols]


    return df_exp

In [8]:
lote001 = pd.read_parquet('./app10/resultado-api/lotes/lote_001.parquet')

lote001.head()

,classe,numeroProcesso,sistema,formato,tribunal,dataHoraUltimaAtualizacao,grau,@timestamp,dataAjuizamento,movimentos,id,nivelSigilo,orgaoJulgador,assuntos,_id,data_download
0,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15019982920248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2025-01-10T19:07:09.830Z,JE,2025-02-07T18:00:06.171637952Z,2024-07-16T17:11:05.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15019982920248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3386, 'nome': 'Leve'}]",TJSP_JE_15019982920248260408,2025-07-04 00:43:31
1,"{'codigo': 10944, 'nome': 'Ação Penal - Proced...",15020035120248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2024-11-13T08:44:38.961Z,JE,2025-02-06T04:58:44.932854109Z,2024-07-16T17:22:38.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15020035120248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3475, 'nome': 'Abandono Intelectua...",TJSP_JE_15020035120248260408,2025-07-04 00:43:31
2,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15019991420248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2024-08-27T01:40:01.997Z,JE,2025-06-16T19:08:53.811380883Z,2024-07-16T17:13:45.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15019991420248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3402, 'nome': 'Ameaça '}]",TJSP_JE_15019991420248260408,2025-07-04 00:43:31
3,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15020121320248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2024-08-27T01:40:13.300Z,JE,2025-06-16T19:08:56.395130508Z,2024-07-17T16:02:53.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15020121320248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3402, 'nome': 'Ameaça '}]",TJSP_JE_15020121320248260408,2025-07-04 00:43:31
4,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15019142820248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2024-08-27T01:40:36.496Z,JE,2025-06-16T19:09:05.169414101Z,2024-07-05T11:10:02.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15019142820248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3402, 'nome': 'Ameaça '}]",TJSP_JE_15019142820248260408,2025-07-04 00:43:31


In [12]:
[lote001.assuntos[i] for i in range(len(lote001))]

["[{'codigo': 3386, 'nome': 'Leve'}]",
 "[{'codigo': 3475, 'nome': 'Abandono Intelectual'}]",
 "[{'codigo': 3402, 'nome': 'Ameaça '}]",
 "[{'codigo': 3402, 'nome': 'Ameaça '}]",
 "[{'codigo': 3402, 'nome': 'Ameaça '}]",
 "[{'codigo': 3435, 'nome': 'Receptação'}]",
 "[{'codigo': 3402, 'nome': 'Ameaça '}]",
 "[{'codigo': 3632, 'nome': 'Crimes de Trânsito'}]",
 "[{'codigo': 3632, 'nome': 'Crimes de Trânsito'}]",
 "[{'codigo': 3572, 'nome': 'Desobediência '}]",
 "[{'codigo': 3402, 'nome': 'Ameaça '}]",
 "[{'codigo': 3632, 'nome': 'Crimes de Trânsito'}]",
 "[{'codigo': 3386, 'nome': 'Leve'}]",
 "[{'codigo': 3395, 'nome': 'Calúnia'}]",
 "[{'codigo': 3397, 'nome': 'Injúria'}]",
 "[{'codigo': 3386, 'nome': 'Leve'}]",
 "[{'codigo': 3402, 'nome': 'Ameaça '}]",
 "[{'codigo': 3431, 'nome': 'Estelionato'}]",
 "[{'codigo': 3692, 'nome': 'Contravenções Penais'}]",
 "[{'codigo': 3692, 'nome': 'Contravenções Penais'}]",
 "[{'codigo': 3397, 'nome': 'Injúria'}]",
 "[{'codigo': 3692, 'nome': 'Contravençõe

In [9]:
colunas = ['classe', 'sistema', 'formato', 'orgaoJulgador', 'assuntos']
df_expandido = expandir_colunas(lote001, colunas)

df_expandido.head()

,numeroProcesso,classe,classe_codigo,classe_nome,sistema,sistema_codigo,sistema_nome,formato,formato_codigo,formato_nome,...,id,nivelSigilo,orgaoJulgador,orgaoJulgador_codigo,orgaoJulgador_nome,assuntos,assuntos_codigo,assuntos_nome,_id,data_download
0,15019982920248260408,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",278,Termo Circunstanciado,"{'codigo': 3, 'nome': 'SAJ'}",3,SAJ,"{'codigo': 1, 'nome': 'Eletrônico'}",1,Eletrônico,...,TJSP_JE_15019982920248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...",10002,01 CRIMINAL DE OURINHOS,"[{'codigo': 3386, 'nome': 'Leve'}]",3386,Leve,TJSP_JE_15019982920248260408,2025-07-04 00:43:31
1,15020035120248260408,"{'codigo': 10944, 'nome': 'Ação Penal - Proced...",10944,Ação Penal - Procedimento Sumaríssimo,"{'codigo': 3, 'nome': 'SAJ'}",3,SAJ,"{'codigo': 1, 'nome': 'Eletrônico'}",1,Eletrônico,...,TJSP_JE_15020035120248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...",10002,01 CRIMINAL DE OURINHOS,"[{'codigo': 3475, 'nome': 'Abandono Intelectua...",3475,Abandono Intelectual,TJSP_JE_15020035120248260408,2025-07-04 00:43:31
2,15019991420248260408,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",278,Termo Circunstanciado,"{'codigo': 3, 'nome': 'SAJ'}",3,SAJ,"{'codigo': 1, 'nome': 'Eletrônico'}",1,Eletrônico,...,TJSP_JE_15019991420248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...",10002,01 CRIMINAL DE OURINHOS,"[{'codigo': 3402, 'nome': 'Ameaça '}]",3402,Ameaça,TJSP_JE_15019991420248260408,2025-07-04 00:43:31
3,15020121320248260408,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",278,Termo Circunstanciado,"{'codigo': 3, 'nome': 'SAJ'}",3,SAJ,"{'codigo': 1, 'nome': 'Eletrônico'}",1,Eletrônico,...,TJSP_JE_15020121320248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...",10002,01 CRIMINAL DE OURINHOS,"[{'codigo': 3402, 'nome': 'Ameaça '}]",3402,Ameaça,TJSP_JE_15020121320248260408,2025-07-04 00:43:31
4,15019142820248260408,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",278,Termo Circunstanciado,"{'codigo': 3, 'nome': 'SAJ'}",3,SAJ,"{'codigo': 1, 'nome': 'Eletrônico'}",1,Eletrônico,...,TJSP_JE_15019142820248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...",10002,01 CRIMINAL DE OURINHOS,"[{'codigo': 3402, 'nome': 'Ameaça '}]",3402,Ameaça,TJSP_JE_15019142820248260408,2025-07-04 00:43:31


In [10]:
df_expandido.columns

Index(['numeroProcesso', 'classe', 'classe_codigo', 'classe_nome', 'sistema',
       'sistema_codigo', 'sistema_nome', 'formato', 'formato_codigo',
       'formato_nome', 'tribunal', 'dataHoraUltimaAtualizacao', 'grau',
       '@timestamp', 'dataAjuizamento', 'movimentos', 'id', 'nivelSigilo',
       'orgaoJulgador', 'orgaoJulgador_codigo', 'orgaoJulgador_nome',
       'assuntos', 'assuntos_codigo', 'assuntos_nome', '_id', 'data_download'],
      dtype='object')

In [11]:
df_expandido[df_expandido['numeroProcesso'] == '10018662020248260411'][['assuntos','assuntos_codigo','assuntos_nome']]

,assuntos,assuntos_codigo,assuntos_nome
78,"[{'codigo': 7698, 'nome': 'Perdas e Danos'}, {...",7698|10338,Perdas e Danos|Gratificações e Adicionais


In [12]:
lote001.head()

,classe,numeroProcesso,sistema,formato,tribunal,dataHoraUltimaAtualizacao,grau,@timestamp,dataAjuizamento,movimentos,id,nivelSigilo,orgaoJulgador,assuntos,_id,data_download
0,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15019982920248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2025-01-10T19:07:09.830Z,JE,2025-02-07T18:00:06.171637952Z,2024-07-16T17:11:05.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15019982920248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3386, 'nome': 'Leve'}]",TJSP_JE_15019982920248260408,2025-07-04 00:43:31
1,"{'codigo': 10944, 'nome': 'Ação Penal - Proced...",15020035120248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2024-11-13T08:44:38.961Z,JE,2025-02-06T04:58:44.932854109Z,2024-07-16T17:22:38.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15020035120248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3475, 'nome': 'Abandono Intelectua...",TJSP_JE_15020035120248260408,2025-07-04 00:43:31
2,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15019991420248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2024-08-27T01:40:01.997Z,JE,2025-06-16T19:08:53.811380883Z,2024-07-16T17:13:45.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15019991420248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3402, 'nome': 'Ameaça '}]",TJSP_JE_15019991420248260408,2025-07-04 00:43:31
3,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15020121320248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2024-08-27T01:40:13.300Z,JE,2025-06-16T19:08:56.395130508Z,2024-07-17T16:02:53.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15020121320248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3402, 'nome': 'Ameaça '}]",TJSP_JE_15020121320248260408,2025-07-04 00:43:31
4,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15019142820248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2024-08-27T01:40:36.496Z,JE,2025-06-16T19:09:05.169414101Z,2024-07-05T11:10:02.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15019142820248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3402, 'nome': 'Ameaça '}]",TJSP_JE_15019142820248260408,2025-07-04 00:43:31


In [13]:
lote001['movimentos'][0]

array([{'codigo': 26, 'complementosTabelados': array([{'codigo': 2, 'descricao': 'tipo_de_distribuicao_redistribuicao', 'nome': 'sorteio', 'valor': 2}],
             dtype=object), 'dataHora': '2024-07-16T17:12:02.000Z', 'nome': 'Distribuição'}                                                             ,
       {'codigo': 581, 'complementosTabelados': array([{'codigo': 4, 'descricao': 'tipo_de_documento', 'nome': 'Outros documentos', 'valor': 80}],
             dtype=object), 'dataHora': '2024-07-16T17:18:33.000Z', 'nome': 'Documento'}                                                          ,
       {'codigo': 11383, 'complementosTabelados': None, 'dataHora': '2024-07-17T10:22:21.000Z', 'nome': 'Ato ordinatório'},
       {'codigo': 60, 'complementosTabelados': array([{'codigo': 4, 'descricao': 'tipo_de_documento', 'nome': 'Certidão', 'valor': 107}],
             dtype=object), 'dataHora': '2024-07-17T10:22:36.000Z', 'nome': 'Expedição de documento'}                                   

In [20]:
def extrair_movimentos_completos(df: pd.DataFrame) -> pd.DataFrame:
    todos_movimentos = []

    for _, row in df.iterrows():
        processo_id = row.get("_id")
        numero_processo = row.get("numeroProcesso")
        movimentos = row.get("movimentos")
        data_download = row.get("data_download")

        if not isinstance(movimentos, (list, np.ndarray)):
            continue

        for mov in movimentos:
            movimento_info = {
                "_id": processo_id,
                "numeroProcesso": numero_processo,
                "movimentos_codigo": mov.get("codigo"),
                "movimentos_nome": mov.get("nome"),
                "movimentos_dataHora": mov.get("dataHora"),
                "movimentos_orgaoJulgador": mov.get("orgaoJulgador"),
                "movimentos_orgaoJulgador_codigoOrgao": None,
                "movimentos_orgaoJulgador_nomeOrgao": None,
                "movimentos_complementosTabelados": None,
                "movimentos_complementosTabelados_codigo": None,
                "movimentos_complementosTabelados_descricao": None,
                "movimentos_complementosTabelados_valor": None,
                "movimentos_complementosTabelados_nome": None,
                "data_download": data_download
            }

            # Orgao julgador (dict)
            orgao = mov.get("orgaoJulgador")
            if isinstance(orgao, dict):
                movimento_info["movimentos_orgaoJulgador_codigoOrgao"] = orgao.get("codigoOrgao")
                movimento_info["movimentos_orgaoJulgador_nomeOrgao"] = orgao.get("nomeOrgao")

            # Complementos Tabelados (list ou ndarray de dicts)
            complementos = mov.get("complementosTabelados")

            if isinstance(complementos, np.ndarray):
                complementos = complementos.tolist()

            if isinstance(complementos, list) and len(complementos) > 0:
                for comp in complementos:
                    comp_info = movimento_info.copy()
                    comp_info["movimentos_complementosTabelados"] = complementos
                    comp_info["movimentos_complementosTabelados_codigo"] = comp.get("codigo")
                    comp_info["movimentos_complementosTabelados_descricao"] = comp.get("descricao")
                    comp_info["movimentos_complementosTabelados_valor"] = comp.get("valor")
                    comp_info["movimentos_complementosTabelados_nome"] = comp.get("nome")
                    todos_movimentos.append(comp_info)
            else:
                # Sem complementos, mas ainda queremos registrar o movimento
                todos_movimentos.append(movimento_info)

    df_movimentos = pd.DataFrame(todos_movimentos)

    if not df_movimentos.empty and "movimentos_dataHora" in df_movimentos.columns:
        df_movimentos["movimentos_dataHora"] = pd.to_datetime(df_movimentos["movimentos_dataHora"], errors="coerce")
        ordem_id_original = pd.CategoricalDtype(categories = df['_id'].to_list(),ordered=True)
        df_movimentos['_id'] = df_movimentos['_id'].astype(ordem_id_original)

        df_movimentos = df_movimentos.sort_values(by=['_id', "movimentos_dataHora"], ascending=[True,False])
        
        df_movimentos = df_movimentos.reset_index(drop=True)
    
    return df_movimentos


In [15]:
lote001.iloc[[0]]

,classe,numeroProcesso,sistema,formato,tribunal,dataHoraUltimaAtualizacao,grau,@timestamp,dataAjuizamento,movimentos,id,nivelSigilo,orgaoJulgador,assuntos,_id,data_download
0,"{'codigo': 278, 'nome': 'Termo Circunstanciado'}",15019982920248260408,"{'codigo': 3, 'nome': 'SAJ'}","{'codigo': 1, 'nome': 'Eletrônico'}",TJSP,2025-01-10T19:07:09.830Z,JE,2025-02-07T18:00:06.171637952Z,2024-07-16T17:11:05.000Z,"[{'codigo': 26, 'complementosTabelados': [{'co...",TJSP_JE_15019982920248260408,0,"{'codigo': 10002, 'nome': '01 CRIMINAL DE OURI...","[{'codigo': 3386, 'nome': 'Leve'}]",TJSP_JE_15019982920248260408,2025-07-04 00:43:31


In [55]:
lote001.iloc[[0]].movimentos[0]

array([{'codigo': 26, 'complementosTabelados': array([{'codigo': 2, 'descricao': 'tipo_de_distribuicao_redistribuicao', 'nome': 'sorteio', 'valor': 2}],
             dtype=object), 'dataHora': '2024-07-16T17:12:02.000Z', 'nome': 'Distribuição'}                                                             ,
       {'codigo': 581, 'complementosTabelados': array([{'codigo': 4, 'descricao': 'tipo_de_documento', 'nome': 'Outros documentos', 'valor': 80}],
             dtype=object), 'dataHora': '2024-07-16T17:18:33.000Z', 'nome': 'Documento'}                                                          ,
       {'codigo': 11383, 'complementosTabelados': None, 'dataHora': '2024-07-17T10:22:21.000Z', 'nome': 'Ato ordinatório'},
       {'codigo': 60, 'complementosTabelados': array([{'codigo': 4, 'descricao': 'tipo_de_documento', 'nome': 'Certidão', 'valor': 107}],
             dtype=object), 'dataHora': '2024-07-17T10:22:36.000Z', 'nome': 'Expedição de documento'}                                   

In [21]:
df_movs = extrair_movimentos_completos(lote001)
df_movs.head(10)


,_id,numeroProcesso,movimentos_codigo,movimentos_nome,movimentos_dataHora,movimentos_orgaoJulgador,movimentos_orgaoJulgador_codigoOrgao,movimentos_orgaoJulgador_nomeOrgao,movimentos_complementosTabelados,movimentos_complementosTabelados_codigo,movimentos_complementosTabelados_descricao,movimentos_complementosTabelados_valor,movimentos_complementosTabelados_nome,data_download
0,TJSP_JE_15019982920248260408,15019982920248260408,60,Expedição de documento,2024-12-17 16:45:31+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão,2025-07-04 00:43:31
1,TJSP_JE_15019982920248260408,15019982920248260408,11010,Mero expediente,2024-12-17 16:36:05+00:00,None,None,None,None,NaN,None,NaN,None,2025-07-04 00:43:31
2,TJSP_JE_15019982920248260408,15019982920248260408,51,Conclusão,2024-12-17 16:29:46+00:00,None,None,None,"[{'codigo': 3, 'descricao': 'tipo_de_conclusao...",3.0,tipo_de_conclusao,5.0,para despacho,2025-07-04 00:43:31
3,TJSP_JE_15019982920248260408,15019982920248260408,51,Conclusão,2024-12-17 16:02:56+00:00,None,None,None,"[{'codigo': 3, 'descricao': 'tipo_de_conclusao...",3.0,tipo_de_conclusao,5.0,para despacho,2025-07-04 00:43:31
4,TJSP_JE_15019982920248260408,15019982920248260408,85,Petição,2024-12-17 15:54:48+00:00,None,None,None,"[{'codigo': 19, 'descricao': 'tipo_de_peticao'...",19.0,tipo_de_peticao,57.0,Petição (outras),2025-07-04 00:43:31
5,TJSP_JE_15019982920248260408,15019982920248260408,60,Expedição de documento,2024-12-12 16:52:29+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão,2025-07-04 00:43:31
6,TJSP_JE_15019982920248260408,15019982920248260408,11383,Ato ordinatório,2024-12-12 16:52:05+00:00,None,None,None,None,NaN,None,NaN,None,2025-07-04 00:43:31
7,TJSP_JE_15019982920248260408,15019982920248260408,581,Documento,2024-12-12 16:51:17+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos,2025-07-04 00:43:31
8,TJSP_JE_15019982920248260408,15019982920248260408,581,Documento,2024-12-12 16:50:40+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos,2025-07-04 00:43:31
9,TJSP_JE_15019982920248260408,15019982920248260408,581,Documento,2024-12-12 16:49:59+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos,2025-07-04 00:43:31


In [17]:
df_movs[df_movs['_id'] == 'TJSP_JE_15019982920248260408']

,_id,numeroProcesso,movimentos_codigo,movimentos_nome,movimentos_dataHora,movimentos_orgaoJulgador,movimentos_orgaoJulgador_codigoOrgao,movimentos_orgaoJulgador_nomeOrgao,movimentos_complementosTabelados,movimentos_complementosTabelados_codigo,movimentos_complementosTabelados_descricao,movimentos_complementosTabelados_valor,movimentos_complementosTabelados_nome
0,TJSP_JE_15019982920248260408,15019982920248260408,60,Expedição de documento,2024-12-17 16:45:31+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão
1,TJSP_JE_15019982920248260408,15019982920248260408,11010,Mero expediente,2024-12-17 16:36:05+00:00,None,None,None,None,NaN,None,NaN,None
2,TJSP_JE_15019982920248260408,15019982920248260408,51,Conclusão,2024-12-17 16:29:46+00:00,None,None,None,"[{'codigo': 3, 'descricao': 'tipo_de_conclusao...",3.0,tipo_de_conclusao,5.0,para despacho
3,TJSP_JE_15019982920248260408,15019982920248260408,51,Conclusão,2024-12-17 16:02:56+00:00,None,None,None,"[{'codigo': 3, 'descricao': 'tipo_de_conclusao...",3.0,tipo_de_conclusao,5.0,para despacho
4,TJSP_JE_15019982920248260408,15019982920248260408,85,Petição,2024-12-17 15:54:48+00:00,None,None,None,"[{'codigo': 19, 'descricao': 'tipo_de_peticao'...",19.0,tipo_de_peticao,57.0,Petição (outras)
5,TJSP_JE_15019982920248260408,15019982920248260408,60,Expedição de documento,2024-12-12 16:52:29+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão
6,TJSP_JE_15019982920248260408,15019982920248260408,11383,Ato ordinatório,2024-12-12 16:52:05+00:00,None,None,None,None,NaN,None,NaN,None
7,TJSP_JE_15019982920248260408,15019982920248260408,581,Documento,2024-12-12 16:51:17+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos
8,TJSP_JE_15019982920248260408,15019982920248260408,581,Documento,2024-12-12 16:50:40+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos
9,TJSP_JE_15019982920248260408,15019982920248260408,581,Documento,2024-12-12 16:49:59+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos


In [52]:
df_movs.groupby('_id').size().reset_index(name='quantidade_movimentos').sort_values(by='quantidade_movimentos', ascending=False)


C:\Users\mooliveira\AppData\Local\Temp\ipykernel_12352\2319662365.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_movs.groupby('_id').size().reset_index(name='quantidade_movimentos').sort_values(by='quantidade_movimentos', ascending=False)


,_id,quantidade_movimentos
17,TJSP_G1_15007291220248260583,149
49,TJSP_G1_15002115320248260411,108
69,TJSP_G1_10018090220248260411,60
10,TJSP_JE_15020529220248260408,58
5,TJSP_JE_15020372620248260408,55
68,TJSP_G1_10018402220248260411,51
57,TJSP_G1_10018280820248260411,51
11,TJSP_JE_15018761620248260408,51
48,TJSP_JE_15002115320248260411,49
34,TJSP_JE_15020806020248260408,48


In [125]:
lote001['movimentos'][0]

array([{'codigo': 26, 'complementosTabelados': array([{'codigo': 2, 'descricao': 'tipo_de_distribuicao_redistribuicao', 'nome': 'sorteio', 'valor': 2}],
             dtype=object), 'dataHora': '2024-07-16T17:12:02.000Z', 'nome': 'Distribuição'}                                                             ,
       {'codigo': 581, 'complementosTabelados': array([{'codigo': 4, 'descricao': 'tipo_de_documento', 'nome': 'Outros documentos', 'valor': 80}],
             dtype=object), 'dataHora': '2024-07-16T17:18:33.000Z', 'nome': 'Documento'}                                                          ,
       {'codigo': 11383, 'complementosTabelados': None, 'dataHora': '2024-07-17T10:22:21.000Z', 'nome': 'Ato ordinatório'},
       {'codigo': 60, 'complementosTabelados': array([{'codigo': 4, 'descricao': 'tipo_de_documento', 'nome': 'Certidão', 'valor': 107}],
             dtype=object), 'dataHora': '2024-07-17T10:22:36.000Z', 'nome': 'Expedição de documento'}                                   

In [54]:
df_movs[df_movs['_id'] == 'TJSP_JE_15019982920248260408'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 27 to 0
Data columns (total 12 columns):
 #   Column                                      Non-Null Count  Dtype              
---  ------                                      --------------  -----              
 0   _id                                         28 non-null     object             
 1   movimentos_codigo                           28 non-null     int64              
 2   movimentos_nome                             28 non-null     object             
 3   movimentos_dataHora                         28 non-null     datetime64[ns, UTC]
 4   movimentos_orgaoJulgador                    0 non-null      object             
 5   movimentos_orgaoJulgador_codigoOrgao        0 non-null      object             
 6   movimentos_orgaoJulgador_nomeOrgao          0 non-null      object             
 7   movimentos_complementosTabelados            22 non-null     object             
 8   movimentos_complementosTabelados_codigo     22 

In [58]:
lote001[lote001['_id'] == 'TJSP_JE_15019982920248260408'][['movimentos']]

,movimentos
0,"[{'codigo': 26, 'complementosTabelados': [{'co..."


In [44]:
df_movs[df_movs['_id'] == 'TJSP_JE_15019982920248260408']

,_id,movimentos_codigo,movimentos_nome,movimentos_dataHora,movimentos_orgaoJulgador,movimentos_orgaoJulgador_codigoOrgao,movimentos_orgaoJulgador_nomeOrgao,movimentos_complementosTabelados,movimentos_complementosTabelados_codigo,movimentos_complementosTabelados_descricao,movimentos_complementosTabelados_valor,movimentos_complementosTabelados_nome
27,TJSP_JE_15019982920248260408,60,Expedição de documento,2024-12-17 16:45:31+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão
26,TJSP_JE_15019982920248260408,11010,Mero expediente,2024-12-17 16:36:05+00:00,None,None,None,None,NaN,None,NaN,None
25,TJSP_JE_15019982920248260408,51,Conclusão,2024-12-17 16:29:46+00:00,None,None,None,"[{'codigo': 3, 'descricao': 'tipo_de_conclusao...",3.0,tipo_de_conclusao,5.0,para despacho
24,TJSP_JE_15019982920248260408,51,Conclusão,2024-12-17 16:02:56+00:00,None,None,None,"[{'codigo': 3, 'descricao': 'tipo_de_conclusao...",3.0,tipo_de_conclusao,5.0,para despacho
23,TJSP_JE_15019982920248260408,85,Petição,2024-12-17 15:54:48+00:00,None,None,None,"[{'codigo': 19, 'descricao': 'tipo_de_peticao'...",19.0,tipo_de_peticao,57.0,Petição (outras)
22,TJSP_JE_15019982920248260408,60,Expedição de documento,2024-12-12 16:52:29+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão
21,TJSP_JE_15019982920248260408,11383,Ato ordinatório,2024-12-12 16:52:05+00:00,None,None,None,None,NaN,None,NaN,None
20,TJSP_JE_15019982920248260408,581,Documento,2024-12-12 16:51:17+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos
19,TJSP_JE_15019982920248260408,581,Documento,2024-12-12 16:50:40+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos
18,TJSP_JE_15019982920248260408,581,Documento,2024-12-12 16:49:59+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,80.0,Outros documentos


### DEPURANDO FUNÇÃO PARA EXTRAÇÃO DOS MOVIMENTOS

In [88]:
df_movs.shape

(2673, 12)

#### 1. Verificar colunas esperadas

In [59]:
colunas_esperadas = [
    "_id",
    "movimentos_codigo",
    "movimentos_nome",
    "movimentos_dataHora",
    "movimentos_complementosTabelados",
    "movimentos_complementosTabelados_codigo",
    "movimentos_complementosTabelados_descricao",
    "movimentos_complementosTabelados_valor",
    "movimentos_complementosTabelados_nome",
    "movimentos_orgaoJulgador",
    "movimentos_orgaoJulgador_codigoOrgao",
    "movimentos_orgaoJulgador_nomeOrgao",
]

colunas_faltantes = set(colunas_esperadas) - set(df_movs.columns)

print("Colunas faltantes:", colunas_faltantes if colunas_faltantes else "Nenhuma")



Colunas faltantes: Nenhuma


#### 2. Verificar tipos de dados


In [80]:
print(df_movs.dtypes)


_id                                                        object
movimentos_codigo                                           int64
movimentos_nome                                            object
movimentos_dataHora                           datetime64[ns, UTC]
movimentos_orgaoJulgador                                   object
movimentos_orgaoJulgador_codigoOrgao                       object
movimentos_orgaoJulgador_nomeOrgao                         object
movimentos_complementosTabelados                           object
movimentos_complementosTabelados_codigo                   float64
movimentos_complementosTabelados_descricao                 object
movimentos_complementosTabelados_valor                    float64
movimentos_complementosTabelados_nome                      object
dtype: object


#### 3. Inspecionar valores não nulos por coluna

- Checar se algum campo está completamente vazio

In [63]:
print(df_movs.notnull().sum())


_id                                           2673
movimentos_codigo                             2673
movimentos_nome                               2673
movimentos_dataHora                           2673
movimentos_orgaoJulgador                         0
movimentos_orgaoJulgador_codigoOrgao             0
movimentos_orgaoJulgador_nomeOrgao               0
movimentos_complementosTabelados              1927
movimentos_complementosTabelados_codigo       1927
movimentos_complementosTabelados_descricao    1927
movimentos_complementosTabelados_valor        1927
movimentos_complementosTabelados_nome         1927
dtype: int64


#### 4. Exibir exemplos onde complementos foram extraídos

In [81]:
df_movs[df_movs["movimentos_complementosTabelados"].notnull()]


,_id,movimentos_codigo,movimentos_nome,movimentos_dataHora,movimentos_orgaoJulgador,movimentos_orgaoJulgador_codigoOrgao,movimentos_orgaoJulgador_nomeOrgao,movimentos_complementosTabelados,movimentos_complementosTabelados_codigo,movimentos_complementosTabelados_descricao,movimentos_complementosTabelados_valor,movimentos_complementosTabelados_nome
720,TJSP_JE_15020009620248260408,85,Petição,2025-01-31 19:23:08+00:00,None,None,None,"[{'codigo': 19, 'descricao': 'tipo_de_peticao'...",19.0,tipo_de_peticao,57.0,Petição (outras)
317,TJSP_JE_15020529220248260408,60,Expedição de documento,2025-01-29 09:45:00+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,79.0,Ofício
316,TJSP_JE_15020529220248260408,60,Expedição de documento,2025-01-29 09:43:59+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,79.0,Ofício
313,TJSP_JE_15020529220248260408,60,Expedição de documento,2025-01-28 14:24:36+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão
719,TJSP_JE_15020009620248260408,106,Mandado,2025-01-28 11:35:16+00:00,None,None,None,"[{'codigo': 6, 'descricao': 'resultado', 'nome...",6.0,resultado,7.0,entregue ao destinatário
...,...,...,...,...,...,...,...,...,...,...,...,...
1344,TJSP_G1_15002115320248260411,85,Petição,2024-04-18 11:41:18+00:00,None,None,None,"[{'codigo': 19, 'descricao': 'tipo_de_peticao'...",19.0,tipo_de_peticao,57.0,Petição (outras)
1343,TJSP_G1_15002115320248260411,60,Expedição de documento,2024-04-17 10:06:27+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão
1294,TJSP_JE_15002115320248260411,60,Expedição de documento,2024-04-17 10:06:27+00:00,None,None,None,"[{'codigo': 4, 'descricao': 'tipo_de_documento...",4.0,tipo_de_documento,107.0,Certidão
1292,TJSP_JE_15002115320248260411,26,Distribuição,2024-04-16 13:57:22+00:00,None,None,None,"[{'codigo': 2, 'descricao': 'tipo_de_distribui...",2.0,tipo_de_distribuicao_redistribuicao,2.0,sorteio


#### 5. Verificar valores únicos para campos categóricos

In [73]:
print(df_movs["movimentos_nome"].unique())
print(df_movs["movimentos_complementosTabelados_nome"].unique())


['Petição' 'Expedição de documento' 'Definitivo' 'Ato ordinatório'
 'Mandado' 'Documento' 'Decadência ou perempção' 'Conclusão' 'Publicação'
 'Mero expediente' 'Remessa' 'Decurso de Prazo' 'Protocolo de Petição'
 'Determinação de arquivamento de procedimentos investigatórios'
 'Cumprimento de transação penal' 'Preliminar' 'Transação Penal'
 'Execução/Cumprimento de Sentença Iniciada (o)' 'Trânsito em julgado'
 'Recebimento' 'Perempção, litispendência ou coisa julgada'
 'Outras Decisões' 'de Instrução e Julgamento' 'Baixa Definitiva'
 'Evolução da Classe Processual' 'Gratuidade da Justiça' 'Procedência'
 'Apensamento' 'Renúncia do queixoso ou perdão aceito '
 'Julgamento em Diligência' 'Desmembramento de Feitos' 'Improcedência'
 'Liberdade Provisória' 'Não Acolhimento de Embargos de Declaração'
 'Desistência' 'Denúncia' 'Redistribuição'
 'Indeferimento da petição inicial' 'Distribuição' 'Procedência em Parte'
 'Prisão em Flagrante em Prisão Preventiva' 'Liberdade provisória'
 'Antecipaç

#### 6. Verificar casos com dados aninhados incompletos ou malformados

In [74]:
df_movs[
    df_movs["movimentos_complementosTabelados"].notnull() &
    df_movs["movimentos_complementosTabelados_codigo"].isnull()
]


,_id,movimentos_codigo,movimentos_nome,movimentos_dataHora,movimentos_orgaoJulgador,movimentos_orgaoJulgador_codigoOrgao,movimentos_orgaoJulgador_nomeOrgao,movimentos_complementosTabelados,movimentos_complementosTabelados_codigo,movimentos_complementosTabelados_descricao,movimentos_complementosTabelados_valor,movimentos_complementosTabelados_nome


####  7. Contar registros totais e com complementos

In [75]:
print("Total de movimentos extraídos:", len(df_movs))
print("Com complementos:", df_movs["movimentos_complementosTabelados"].notnull().sum())


Total de movimentos extraídos: 2673
Com complementos: 1927


#### 8. Verificar se há valores inconsistentes

In [76]:
df_movs[
    df_movs["movimentos_complementosTabelados"].notnull() &
    df_movs["movimentos_complementosTabelados_codigo"].isnull() &
    df_movs["movimentos_complementosTabelados_nome"].isnull()
]


,_id,movimentos_codigo,movimentos_nome,movimentos_dataHora,movimentos_orgaoJulgador,movimentos_orgaoJulgador_codigoOrgao,movimentos_orgaoJulgador_nomeOrgao,movimentos_complementosTabelados,movimentos_complementosTabelados_codigo,movimentos_complementosTabelados_descricao,movimentos_complementosTabelados_valor,movimentos_complementosTabelados_nome


### Variação extração movimentos 

Extração dos campos

- movimentos.orgaoJulgador	
- movimentos.orgaoJulgador.codigoOrgao
- movimentos.orgaoJulgador.nomeOrgao	

In [83]:
def extrair_movimentos2(df: pd.DataFrame) -> pd.DataFrame:
    """Extrai campos dos movimentos processuais, incluindo orgaoJulgador e complementosTabelados."""

    movimentos = df[["_id", "movimentos"]].explode("movimentos").reset_index(drop=True)

    movimentos["movimentos_codigo"] = movimentos["movimentos"].apply(lambda x: x.get("codigo") if isinstance(x, dict) else None)
    movimentos["movimentos_nome"] = movimentos["movimentos"].apply(lambda x: x.get("nome") if isinstance(x, dict) else None)
    movimentos["movimentos_dataHora"] = movimentos["movimentos"].apply(lambda x: x.get("dataHora") if isinstance(x, dict) else None)

    # ✅ Extraindo o dicionário bruto do orgaoJulgador
    movimentos["movimentos_orgaoJulgador"] = movimentos["movimentos"].apply(
        lambda x: x.get("orgaoJulgador") if isinstance(x, dict) else None
    )

    # 🔍 Extraindo os subcampos
    movimentos["movimentos_orgaoJulgador_codigoOrgao"] = movimentos["movimentos_orgaoJulgador"].apply(
        lambda x: x.get("codigoOrgao") if isinstance(x, dict) else None
    )
    movimentos["movimentos_orgaoJulgador_nomeOrgao"] = movimentos["movimentos_orgaoJulgador"].apply(
        lambda x: x.get("nomeOrgao") if isinstance(x, dict) else None
    )

    # ✅ ComplementosTabelados
    movimentos["movimentos_complementosTabelados"] = movimentos["movimentos"].apply(
        lambda x: x.get("complementosTabelados") if isinstance(x, dict) else None
    )

    movimentos = movimentos.explode("movimentos_complementosTabelados").reset_index(drop=True)

    movimentos["movimentos_complementosTabelados_codigo"] = movimentos["movimentos_complementosTabelados"].apply(
        lambda x: x.get("codigo") if isinstance(x, dict) else None
    )
    movimentos["movimentos_complementosTabelados_descricao"] = movimentos["movimentos_complementosTabelados"].apply(
        lambda x: x.get("descricao") if isinstance(x, dict) else None
    )
    movimentos["movimentos_complementosTabelados_valor"] = movimentos["movimentos_complementosTabelados"].apply(
        lambda x: x.get("valor") if isinstance(x, dict) else None
    )
    movimentos["movimentos_complementosTabelados_nome"] = movimentos["movimentos_complementosTabelados"].apply(
        lambda x: x.get("nome") if isinstance(x, dict) else None
    )

    # 🎯 Selecionando apenas as colunas desejadas
    colunas_finais = [
        "_id",
        "movimentos_codigo",
        "movimentos_nome",
        "movimentos_dataHora",
        "movimentos_orgaoJulgador",
        "movimentos_orgaoJulgador_codigoOrgao",
        "movimentos_orgaoJulgador_nomeOrgao",
        "movimentos_complementosTabelados",
        "movimentos_complementosTabelados_codigo",
        "movimentos_complementosTabelados_descricao",
        "movimentos_complementosTabelados_valor",
        "movimentos_complementosTabelados_nome"
    ]

    return movimentos[colunas_finais]


In [84]:
df = extrair_movimentos2(lote001)

In [85]:
df

,_id,movimentos_codigo,movimentos_nome,movimentos_dataHora,movimentos_orgaoJulgador,movimentos_orgaoJulgador_codigoOrgao,movimentos_orgaoJulgador_nomeOrgao,movimentos_complementosTabelados,movimentos_complementosTabelados_codigo,movimentos_complementosTabelados_descricao,movimentos_complementosTabelados_valor,movimentos_complementosTabelados_nome
0,TJSP_JE_15019982920248260408,26,Distribuição,2024-07-16T17:12:02.000Z,None,None,None,"{'codigo': 2, 'descricao': 'tipo_de_distribuic...",2.0,tipo_de_distribuicao_redistribuicao,2.0,sorteio
1,TJSP_JE_15019982920248260408,581,Documento,2024-07-16T17:18:33.000Z,None,None,None,"{'codigo': 4, 'descricao': 'tipo_de_documento'...",4.0,tipo_de_documento,80.0,Outros documentos
2,TJSP_JE_15019982920248260408,11383,Ato ordinatório,2024-07-17T10:22:21.000Z,None,None,None,None,NaN,None,NaN,None
3,TJSP_JE_15019982920248260408,60,Expedição de documento,2024-07-17T10:22:36.000Z,None,None,None,"{'codigo': 4, 'descricao': 'tipo_de_documento'...",4.0,tipo_de_documento,107.0,Certidão
4,TJSP_JE_15019982920248260408,85,Petição,2024-07-24T12:56:09.000Z,None,None,None,"{'codigo': 19, 'descricao': 'tipo_de_peticao',...",19.0,tipo_de_peticao,57.0,Petição (outras)
...,...,...,...,...,...,...,...,...,...,...,...,...
2668,TJSP_G1_10018766420248260411,123,Remessa,2024-09-25T05:44:58.000Z,None,None,None,"{'codigo': 18, 'descricao': 'motivo_da_remessa...",18.0,motivo_da_remessa,40.0,outros motivos
2669,TJSP_G1_10018766420248260411,92,Publicação,2024-09-26T01:16:49.000Z,None,None,None,None,NaN,None,NaN,None
2670,TJSP_G1_10018766420248260411,60,Expedição de documento,2024-10-05T07:55:27.000Z,None,None,None,"{'codigo': 4, 'descricao': 'tipo_de_documento'...",4.0,tipo_de_documento,107.0,Certidão
2671,TJSP_G1_10018766420248260411,848,Trânsito em julgado,2024-10-22T15:20:55.000Z,None,None,None,None,NaN,None,NaN,None


In [86]:
df.columns

Index(['_id', 'movimentos_codigo', 'movimentos_nome', 'movimentos_dataHora',
       'movimentos_orgaoJulgador', 'movimentos_orgaoJulgador_codigoOrgao',
       'movimentos_orgaoJulgador_nomeOrgao',
       'movimentos_complementosTabelados',
       'movimentos_complementosTabelados_codigo',
       'movimentos_complementosTabelados_descricao',
       'movimentos_complementosTabelados_valor',
       'movimentos_complementosTabelados_nome'],
      dtype='object')

In [87]:
df.shape

(2673, 12)

In [91]:
print(df.notnull().sum())

_id                                           2673
movimentos_codigo                             2673
movimentos_nome                               2673
movimentos_dataHora                           2673
movimentos_orgaoJulgador                         0
movimentos_orgaoJulgador_codigoOrgao             0
movimentos_orgaoJulgador_nomeOrgao               0
movimentos_complementosTabelados              1927
movimentos_complementosTabelados_codigo       1927
movimentos_complementosTabelados_descricao    1927
movimentos_complementosTabelados_valor        1927
movimentos_complementosTabelados_nome         1927
orgaoJulgador_valido                          2673
dtype: int64


In [92]:
# Verifica se ao menos um movimento possui 'orgaoJulgador'
existe_orgaoJulgador = lote001['movimentos'].explode().dropna().apply(
    lambda x: isinstance(x, dict) and 'orgaoJulgador' in x
).sum()

print(f"Número de movimentos com 'orgaoJulgador': {existe_orgaoJulgador}")


Número de movimentos com 'orgaoJulgador': 0
